In [1]:
import requests
import pandas as pd 
class ScriptData:
    
    def __init__(self, key):
        self.key = key
        self.base_url = "https://www.alphavantage.co/query?"
        self.data ={}
    def __getitem__(self, script):
        if script in self.data:
            return self.data[script]
        else:
            return None   
    def __setitem__(self, key, value):
        self.data[key] = value
    def __contains__(self, key):
        return key in self.data
    
    
    def fetch_intraday_data(self,script) :
        if script in self.data :
            return self.data[script]
        
        parameters = {
            "function" : "TIME_SERIES_INTRADAY",
            "symbol" : script,
            "interval" : "1min",
            "apikey" : self.key
        }
        response = requests.get(self.base_url, params=parameters)
        
        if response.status_code == 200:
            self.data[script]=response.json()
            return self.data[script]
        else:
            return None

        
    def convert_intraday_data(self,script) :
            
        data = self.fetch_intraday_data(script)
        if data is None:
            return None
        
        del data["Meta Data"]
        timestamp_data=list(list())
        for key in data['Time Series (1min)'] :
            temp=[]
            temp.append(key)
            for k in data['Time Series (1min)'][key].values() :
                temp.append(k)
            timestamp_data.append(temp)
    
        columns = ['timestamp','open','high','low','close','volume']
        df = pd.DataFrame(timestamp_data,columns=columns)
        
        del self.data[script]
        self.data[script]=df

In [2]:
key = "RCLRXKEBTO17UXET"
script_data = ScriptData(key)
script_data.fetch_intraday_data("GOOGL")
script_data.convert_intraday_data("GOOGL")
script_data["GOOGL"]

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,94.1800,94.1800,94.1100,94.1200,3222
1,2023-02-14 19:59:00,94.1700,94.1800,94.1700,94.1800,3081
2,2023-02-14 19:58:00,94.1600,94.1800,94.1600,94.1800,1594
3,2023-02-14 19:56:00,94.1800,94.1800,94.1500,94.1500,1277
4,2023-02-14 19:55:00,94.2200,94.2200,94.1800,94.1800,614
...,...,...,...,...,...,...
95,2023-02-14 17:40:00,94.3900,94.3900,94.3500,94.3500,884
96,2023-02-14 17:39:00,94.3900,94.4000,94.3900,94.4000,502
97,2023-02-14 17:38:00,94.3700,94.4000,94.3700,94.4000,408
98,2023-02-14 17:36:00,94.3600,94.3600,94.3600,94.3600,2632


In [3]:
print('GOOGL' in script_data)
print('AAPL' in script_data)

True
False


In [4]:
def indicator1(df,timeperiod) :
    columns = ['timestamp','close']
    col_rename=['timestamp','indicator']
    
    new_df = df[columns].rename(columns = {
        'close': 'indicator',
    }).copy()
    
    new_df['indicator']=new_df['indicator'].rolling(window=timeperiod).mean()
    return new_df
iGoogle=indicator1(script_data["GOOGL"],5)
iGoogle

,timestamp,indicator
0,2023-02-14 20:00:00,NaN
1,2023-02-14 19:59:00,NaN
2,2023-02-14 19:58:00,NaN
3,2023-02-14 19:56:00,NaN
4,2023-02-14 19:55:00,94.162
...,...,...
95,2023-02-14 17:40:00,94.374
96,2023-02-14 17:39:00,94.374
97,2023-02-14 17:38:00,94.378
98,2023-02-14 17:36:00,94.380


In [5]:
class Strategy(ScriptData) :
    def __init__(self,key) :
        super().__init__(key)
        
    def getSignals(self,val,timeperiod) :
        df = self.data[val]
        df1 = indicator1(df,timeperiod)
        print(df)
        

In [6]:
key = "RCLRXKEBTO17UXET"
strategy = Strategy("GOOGL")
strategy.fetch_intraday_data("GOOGL")
strategy.convert_intraday_data("GOOGL")
strategy['GOOGL']
strategy = Strategy("AAPL")
strategy.fetch_intraday_data("AAPL")
strategy.convert_intraday_data("AAPL")
strategy['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,152.8000,152.8500,152.7900,152.8400,5807
1,2023-02-14 19:59:00,152.8000,152.8000,152.7900,152.8000,1037
2,2023-02-14 19:58:00,152.7900,152.8000,152.7600,152.8000,1147
3,2023-02-14 19:57:00,152.7900,152.7900,152.7700,152.7700,1438
4,2023-02-14 19:55:00,152.7300,152.7700,152.7300,152.7700,1616
...,...,...,...,...,...,...
95,2023-02-14 17:46:00,153.1200,153.1200,153.1200,153.1200,898
96,2023-02-14 17:45:00,153.1200,153.1200,153.1200,153.1200,329
97,2023-02-14 17:42:00,153.1200,153.1300,153.1200,153.1300,1278
98,2023-02-14 17:38:00,153.0800,153.0800,153.0800,153.0800,419


In [7]:
strategy.getSignals('AAPL',5)

              timestamp  indicator
0   2023-02-14 20:00:00        NaN
1   2023-02-14 19:59:00        NaN
2   2023-02-14 19:58:00        NaN
3   2023-02-14 19:57:00        NaN
4   2023-02-14 19:55:00  152.79600
..                  ...        ...
95  2023-02-14 17:46:00  153.12600
96  2023-02-14 17:45:00  153.12400
97  2023-02-14 17:42:00  153.12600
98  2023-02-14 17:38:00  153.11600
99  2023-02-14 17:35:00  153.09922

[100 rows x 2 columns]
